# 第8章 文埋め込み

## 8.4 最近傍探索ライブラリ `Faiss` を使った検索

### 8.4.2 `Faiss`を利用した最近傍探索の実装

#### 準備

In [1]:
!pip install datasets faiss-cpu scipy transformers[ja,torch]

#### データセットの読み込みと前処理

In [2]:
from datasets import load_dataset

# Hugging Face Hubのllm-book/jawiki-paragraphsのリポジトリから
# Wikipediaの段落テキストのデータを読み込む
paragraph_dataset = load_dataset(
    "llm-book/jawiki-paragraphs", split="train"
)

/home/studio-lab-user/.conda/envs/llm-book/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 9668476/9668476 [00:19<00:00, 487067.93 examples/s]


In [3]:
# 段落データの形式と事例数を確認する
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 9668476
})


In [4]:
from pprint import pprint

# 段落データの内容を確認する
pprint(paragraph_dataset[0])
pprint(paragraph_dataset[1])

{'html_tag': 'p',
 'id': '5-89167474-0',
 'pageid': 5,
 'paragraph_index': 0,
 'revid': 89167474,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'html_tag': 'p',
 'id': '5-89167474-1',
 'pageid': 5,
 'paragraph_index': 1,
 'revid': 89167474,
 'section': '語源',
 'text': '英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" '
         'も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" '
         'のように唱えられていた。また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。"&" '
         'はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。結果として、アルファベットの復唱の最後は "X, Y, '
         'Z, and per se and" という形になった。この最後のフレーズが繰り返されるうちに "ampersand" '
         'と訛っていき、この言葉は1837年までには英語の一般的な語法となった。',
 'title': 'アンパサンド'}


In [5]:
# 段落データのうち、各記事の最初の段落のみを使うようにする
paragraph_dataset = paragraph_dataset.filter(
    lambda example: example["paragraph_index"] == 0
)

Filter: 100%|██████████| 9668476/9668476 [01:24<00:00, 114426.20 examples/s]


In [6]:
# フィルタリング後の段落データの形式と事例数を確認する
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 1339236
})


In [7]:
# フィルタリング後の段落データの内容を確認する
pprint(paragraph_dataset[0])
pprint(paragraph_dataset[1])

{'html_tag': 'p',
 'id': '5-89167474-0',
 'pageid': 5,
 'paragraph_index': 0,
 'revid': 89167474,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'html_tag': 'p',
 'id': '10-94194440-0',
 'pageid': 10,
 'paragraph_index': 0,
 'revid': 94194440,
 'section': '__LEAD__',
 'text': '言語(げんご)は、狭義には「声による記号の体系」をいう。',
 'title': '言語'}


#### トークナイザとモデルの準備

Hugging Face Hubから読み込む場合

In [8]:
from transformers import AutoModel, AutoTokenizer

# Hugging Face Hubにアップロードされた
# 教師なしSimCSEのトークナイザとエンコーダを読み込む
model_name = "llm-book/bert-base-japanese-v3-unsup-simcse-jawiki"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)

Google ドライブに保存したモデルを読み込む場合

In [ ]:
from google.colab import drive

drive.mount("drive")

Mounted at drive


In [ ]:
!cp -r drive/MyDrive/llm-book/outputs_unsup_simcse .

In [ ]:
from transformers import AutoModel, AutoTokenizer

# ディスクに保存された教師なしSimCSEのトークナイザとエンコーダを読み込む
model_path = "outputs_unsup_simcse/encoder"
tokenizer = AutoTokenizer.from_pretrained(model_path)
encoder = AutoModel.from_pretrained(model_path)

共通の処理

In [9]:
# 読み込んだモデルをGPUのメモリに移動させる
device = "cuda:0"
encoder = encoder.to(device)

#### モデルによる埋め込みの計算

In [10]:
import numpy as np
import torch
import torch.nn.functional as F

def embed_texts(texts: list[str]) -> np.ndarray:
    """SimCSEのモデルを用いてテキストの埋め込みを計算"""
    # テキストにトークナイザを適用
    tokenized_texts = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt",
    ).to(device)

    # トークナイズされたテキストをベクトルに変換
    with torch.inference_mode():
        with torch.cuda.amp.autocast():
            encoded_texts = encoder(
                **tokenized_texts
            ).last_hidden_state[:, 0]

    # ベクトルをNumPyのarrayに変換
    emb = encoded_texts.cpu().numpy().astype(np.float32)
    # ベクトルのノルムが1になるように正規化
    emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)
    return emb

In [11]:
# 段落データのすべての事例に埋め込みを付与する
paragraph_dataset = paragraph_dataset.map(
    lambda examples: {
        "embeddings": list(embed_texts(examples["text"]))
    },
    batched=True,
)

Parameter 'function'=<function <lambda> at 0x7fdee9639fc0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|██████████| 1339236/1339236 [47:57<00:00, 465.49 examples/s]


In [12]:
# 埋め込みを付与した段落データの形式と事例数を確認する
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag', 'embeddings'],
    num_rows: 1339236
})


In [13]:
# 埋め込みを計算した段落データの内容を確認する
pprint(paragraph_dataset[0])

{'embeddings': [0.042540159076452255,
                -0.04196695238351822,
                -0.032314036041498184,
                0.018254535272717476,
                -0.06702621281147003,
                -0.06091688573360443,
                -0.05339366942644119,
                0.00584664149209857,
                0.005609563086181879,
                0.0004761741147376597,
                0.0544019490480423,
                -0.03015177696943283,
                -0.01541396789252758,
                -0.09760468453168869,
                0.031523678451776505,
                0.0070890034548938274,
                0.004207753110677004,
                -0.01840118132531643,
                -0.07030576467514038,
                0.00973212718963623,
                0.006157499272376299,
                -0.03276313096284866,
                -0.00840569194406271,
                -0.023154262453317642,
                0.051235586404800415,
                0.0434039942920208,
              

In [14]:
# 埋め込みを付与した段落データをディスクに保存する
paragraph_dataset.save_to_disk(
    "outputs_unsup_simcse/embedded_paragraphs"
)

Saving the dataset (6/10 shards):  65%|██████▍   | 865544/1339236 [00:22<00:12, 38235.88 examples/s] 


OSError: [Errno 28] No space left on device

#### Google ドライブへの保存

In [ ]:
from google.colab import drive

drive.mount("drive")

In [ ]:
# 保存された段落データをGoogleドライブのフォルダにコピーする
!cp -r outputs_unsup_simcse/embedded_paragraphs drive/MyDrive/llm-book/outputs_unsup_simcse

#### `Faiss` による最近傍探索を試す

In [15]:
import faiss

# ベクトルの次元数をエンコーダの設定値から取り出す
emb_dim = encoder.config.hidden_size
# ベクトルの次元数を指定して空のFaissインデックスを作成する
index = faiss.IndexFlatIP(emb_dim)
# 段落データの"embeddings"フィールドのベクトルからFaissインデックスを構築する
paragraph_dataset.add_faiss_index("embeddings", custom_index=index)

100%|██████████| 1340/1340 [00:10<00:00, 127.14it/s]


Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag', 'embeddings'],
    num_rows: 1339236
})

In [16]:
query_text = "日本語は、主に日本で話されている言語である。"

# 最近傍探索を実行し、類似度上位10件の事例とスコアを取得する
scores, retrieved_examples = paragraph_dataset.get_nearest_examples(
    "embeddings", embed_texts([query_text])[0], k=10
)
# 取得した事例の内容をスコアとともに表示する
titles = retrieved_examples["title"]
texts = retrieved_examples["text"]
for score, title, text in zip(scores, titles, texts):
    print(score, title, text)

0.7834423 日本の言語 日本の言語(にほんのげんご)は、日本の国土で使用されている言語について記述する。日本#言語も参照。
0.7587499 日本語教育 日本語教育(にほんごきょういく)とは、外国語としての日本語、第二言語としての日本語についての教育の総称である。
0.7493928 日本語学 日本語学(にほんごがく)とは、日本語を研究の対象とする学問である。
0.7472434 日本語 日本語(にほんご、にっぽんご、英語: Japanese)は、日本国内や、かつての日本領だった国、そして国外移民や移住者を含む日本人同士の間で使用されている言語。日本は法令によって公用語を規定していないが、法令その他の公用文は全て日本語で記述され、各種法令において日本語を用いることが規定され、学校教育においては「国語」の教科として学習を行う等、事実上、日本国内において唯一の公用語となっている。
0.70455515 国語 (教科) 国語(こくご、英: Japanese Language)は、日本の学校教育における教科の一つ。
0.7030063 和製英語 和製英語(わせいえいご)は、日本語の中で使われる和製外来語の一つで、日本で日本人により作られた、英語の言葉や英語に似ている言葉(固有名詞や商品名などを除く)である。英語圏では別表現をするために理解されなかったり、もしくは、全く異なった解釈をされたりする場合がある。
0.69570327 口語 口語(こうご)とは、普通の日常的な生活の中での会話で用いられる言葉遣いのことである。書記言語で使われる文語と違い、方言と呼ばれる地域差や社会階層などによる言語変種が応じやすく、これらと共通語などを使い分ける状態はダイグロシアと呼ばれる。
0.69446695 ジャパン ジャパン(英語: Japan)は、英語で日本を意味する単語。
0.6910831 日本語学科 日本語学科(にほんごがっか)とは、日本語を教育研究することを目的として大学や専門学校などの高等教育機関に置かれる学科の名称である。
0.6908722 日本語学校 日本語学校(にほんごがっこう)とは、主に日本語を母語としない者を対象として、第二言語・外国語としての日本語教育を実施する機関。日本国内外に存在している。
